# imports

In [65]:
import random

import numpy as np

# open original vcf file

In [67]:
with open("/Users/minhtoo.lin/repos/nf-gwas/tests/input/pipeline/example.vcf", "r") as f:
    vcf_orig = f.readlines()

header_lines_orig = vcf_orig[:6]

# prepare new chromosome header line

In [72]:
# just copy the fixed part
chrom_header_line_base = "#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT"

num_individuals = 50_000
chrom_header_line_chars = [chrom_header_line_base]
for i in range(1, num_individuals + 1):  # 1-indexed
    chrom_header_line_chars.append(f"\t{i}")
chrom_header_line_chars.append("\n")

chrom_header_line_new = "".join(chrom_header_line_chars)

with open("../data/example_big.vcf", "w") as f:
    # write 6 header lines
    f.writelines(header_lines_orig)
    # write new chromosome header line
    f.write(chrom_header_line_new)

# prepare new data lines, row by row - use random.choice()

In [74]:
GT_CHOICES = ["0/0", "0/1", "1/1"]
num_positions = 1_000

# append
with open("../data/example_big.vcf", "a") as f:
    for row_idx in range(1, num_positions + 1):  # 1-indexed, up to num_position rows
        line_base = f"1\t{row_idx}\t{row_idx}\t2\t1\t.\t.\tPR\tGT"
        curr_line_chars = [line_base]
        
        for i in range(num_individuals):
            random_genotype = random.choice(GT_CHOICES)
            curr_line_chars.append(f"\t{random_genotype}")
        curr_line_chars.append("\n")
            
        f.write("".join(curr_line_chars))

# write fake phenotypes file

In [66]:
num_phenotypes_total = 10
num_phenotypes_per_file = 1

assert num_phenotypes_total % num_phenotypes_per_file == 0

header_base = "FID IID"
curr_pheno_idx = 1  # 1-indexed
for file_idx in range(num_phenotypes_total // num_phenotypes_per_file):
    with open(f"../data/phenotype_big_{file_idx}.txt", "w") as f:
        # write header
        header_chars = [header_base]
        for pheno_idx in range(curr_pheno_idx, curr_pheno_idx + num_phenotypes_per_file):
            header_chars.append(f" Y{pheno_idx}")
        header_chars.append("\n")
        f.write("".join(header_chars))

        # draw from normal distribution
        random_nums = np.random.normal(size=(num_individuals, num_phenotypes_per_file))

        # write data rows
        for row_idx in range(1, num_individuals + 1): # 1-indexed
            row_chars = [f"{row_idx} {row_idx}"]
            for pheno_idx in range(num_phenotypes_per_file):
                row_chars.append(f" {random_nums[row_idx - 1][pheno_idx]}")  # 0-indexed
            row_chars.append("\n")
            f.write("".join(row_chars))

    curr_pheno_idx += num_phenotypes_per_file